<a href="https://colab.research.google.com/github/heckmartin/AM1-Trabalho1/blob/main/2020_ENPE_AM_T1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>